In [161]:
# Import relevant packages
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

import hdmpy
import numpy as np
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors

In [162]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import itertools
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from itertools import compress
import statsmodels.api as sm
import statsmodels.formula.api as smf

This notebook contains an example for teaching.

# A Case Study: The Effect of Gun Ownership on Gun-Homicide Rates

We consider the problem of estimating the effect of gun
ownership on the homicide rate. For this purpose, we estimate the following partially
linear model

$$
 Y_{j,t} = \beta D_{j,(t-1)} + g(Z_{j,t}) + \epsilon_{j,t}.
$$

## Data

$Y_{j,t}$ is log homicide rate in county $j$ at time $t$, $D_{j, t-1}$ is log  fraction of suicides committed with a firearm in county $j$ at time $t-1$, which we use as a proxy for gun ownership,  and  $Z_{j,t}$ is a set of demographic and economic characteristics of county $j$ at time $t$. The parameter $\beta$ is the effect of gun ownership on the
homicide rates, controlling for county-level demographic and economic characteristics. 

The sample covers 195 large United States counties between the years 1980 through 1999, giving us 3900 observations.

In [163]:
data1 = pd.read_csv( r"../data/gun_clean.csv" )
data1.shape[1]

415

### Preprocessing

To account for heterogeneity across counties and time trends in  all variables, we remove from them county-specific and time-specific effects in the following preprocessing.

In [164]:
import re

In [172]:
#################################  Find Variable Names from Dataset ########################

def varlist( df = None, type_dataframe = [ 'numeric' , 'categorical' , 'string' ],  pattern = "", exclude = None ):
    varrs = []
    
    if ('numeric' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_numeric_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('categorical' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_categorical_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('string' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_string_dtype , axis = 0 ).to_list() ].tolist()
    
    grepl_result = np.array( [ re.search( pattern , variable ) is not None for variable in df.columns.to_list() ] )
    
    if exclude is None:
        result = list(compress( varrs, grepl_result ) )
    
    else:
        varrs_excluded = np.array( [var in exclude for var in varrs ] )
        and_filter = np.logical_and( ~varrs_excluded ,  grepl_result )
        result = list(compress( varrs, and_filter ) )
    
    return result   

################################# Create Variables ###############################


# Dummy Variables for Year and County Fixed Effects
r = re.compile("X_Jfips")
fixed = list( filter( r.match, data1.columns.to_list() ) )
year = varlist(data1, pattern="X_Tyear")

census = []
census_var = ["^AGE", "^BN", "^BP", "^BZ", "^ED", "^EL","^HI", "^HS", "^INC", "^LF", "^LN", "^PI", "^PO", "^PP", "^PV", "^SPR", "^VS"]

for variable in census_var:
    r = re.compile( variable )
    census = census + list( filter( r.match, data1.columns.to_list() ) )

    
################################ Variables ##################################
# Treatment Variable
d = "logfssl"

# Outcome Variable
y = "logghomr"

# Other Control Variables
X1 = ["logrobr", "logburg", "burg_missing", "robrate_missing"]
X2 = ["newblack", "newfhh", "newmove", "newdens", "newmal"]

#################################  Partial out Fixed Effects ########################

rdata = data1[['CountyCode']]

# Variables to be Partialled-out
varlist = [y] + [d] + X1 + X2 + census

# Partial out Variables in varlist from year and county fixed effect
for var_name in varlist2:
    form = var_name + " ~ " + "+" + " + ".join( year ) + "+" + " + ".join( fixed )
    rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid

<ipython-input-172-100256a64d94>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid


In [167]:
# load dataset
rdata_read = pyreadr.read_r("../data/gun_clean.RData")
data = rdata_read[ 'data' ]
n = data.shape[0]

## We check that our results are equal to R results at 6 decimals

In [174]:
column_names = data.columns.to_list()

for col in column_names:
    result = (data[f'{col}'].round(6) == rdata[f'{col}'].round(6)).sum()
    
    if result == 3900:
        print(f'Column {col} are equal at 6 decimals.')
    else:
        print(f'\n\nColumn {col} are not equal at 6 decimals.')

Column logghomr are equal at 6 decimals.
Column logfssl are equal at 6 decimals.
Column logrobr are equal at 6 decimals.
Column logburg are equal at 6 decimals.
Column burg_missing are equal at 6 decimals.
Column robrate_missing are equal at 6 decimals.
Column newblack are equal at 6 decimals.
Column newfhh are equal at 6 decimals.
Column newmove are equal at 6 decimals.
Column newdens are equal at 6 decimals.
Column newmal are equal at 6 decimals.
Column AGE010D are equal at 6 decimals.
Column AGE050D are equal at 6 decimals.
Column AGE110D are equal at 6 decimals.
Column AGE170D are equal at 6 decimals.
Column AGE180D are equal at 6 decimals.
Column AGE270D are equal at 6 decimals.
Column AGE310D are equal at 6 decimals.
Column AGE320D are equal at 6 decimals.
Column AGE350D are equal at 6 decimals.
Column AGE380D are equal at 6 decimals.
Column AGE410D are equal at 6 decimals.
Column AGE470D are equal at 6 decimals.
Column AGE570D are equal at 6 decimals.
Column AGE640D are equal at

## The effect of gun ownership

### OLS

After preprocessing the data, we first look at simple regression of $Y_{j,t}$ on $D_{j,t-1}$ without controls as a baseline model.

In [133]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

In [146]:
# OLS clustering at the County level
model = "logghomr ~ logfssl"
baseline_ols = smf.ols(model , data=data2).fit().get_robustcov_results(cov_type = "cluster", groups= data2['CountyCode'])
baseline_ols_table = baseline_ols.summary2().tables[1]
baseline_ols_table

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,8.673617e-19,2.895168e-16,0.002996,0.997613,-5.701373e-16,5.718720e-16
logfssl,2.823045e-01,6.481080e-02,4.355825,0.000021,1.544802e-01,4.101287e-01


In [62]:
# Print results 
baseline_ols_table.loc[ 'logfssl' , : ]

Coef.       0.282304
Std.Err.    0.064811
t           4.355825
P>|t|       0.000021
[0.025      0.154480
0.975]      0.410129
Name: logfssl, dtype: float64

The point estimate is $0.282$ with the confidence interval ranging from 0.155 to 0.41. This
suggests that increases in gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% relative
to a trend then the predicted gun homicide rate goes up by 0.28%, without controlling for counties' characteristics.

Since our goal is to estimate the effect of gun ownership after controlling for a rich set county characteristics we next include the controls. First, we estimate the model by ols and then by an array of the modern regression methods using the double machine learning approach.

In [63]:
# define the variables
y = 'logghomr'

data_columns = list(data)
no_relev_col = ['logfssl', 'CountyCode', 'logghomr']

# This gives us: new_list = ['carrot' , 'lemon']
z = [col for col in data_columns if col not in no_relev_col]

In [64]:
Z = '+'.join(z)
X = 'logfssl'
control_formula  = y + '~' + X + "+" + Z 
control_formula

'logghomr~logfssl+logrobr+logburg+burg_missing+robrate_missing+newblack+newfhh+newmove+newdens+newmal+AGE010D+AGE050D+AGE110D+AGE170D+AGE180D+AGE270D+AGE310D+AGE320D+AGE350D+AGE380D+AGE410D+AGE470D+AGE570D+AGE640D+AGE670D+AGE760D+BNK010D+BNK050D+BPS030D+BPS130D+BPS230D+BPS020D+BPS120D+BPS220D+BPS820D+BZA010D+BZA110D+BZA210D+EDU100D+EDU200D+EDU600D+EDU610D+EDU620D+EDU630D+EDU635D+EDU640D+EDU650D+EDU680D+EDU685D+ELE010D+ELE020D+ELE025D+ELE030D+ELE035D+ELE060D+ELE065D+ELE210D+ELE220D+HIS010D+HIS020D+HIS030D+HIS040D+HIS110D+HIS120D+HIS130D+HIS140D+HIS200D+HIS300D+HIS500D+HIS700D+HSD010D+HSD020D+HSD030D+HSD110D+HSD120D+HSD130D+HSD140D+HSD150D+HSD170D+HSD200D+HSD210D+HSD230D+HSD300D+HSD310D+HSG030D+HSG195D+HSG200D+HSG220D+HSG440D+HSG445D+HSG460D+HSG680D+HSG700D+HSD410D+HSD500D+HSD510D+HSD520D+HSD530D+HSD540D+HSD550D+HSD560D+HSD570D+HSD580D+HSD590D+HSD610D+HSD620D+HSD710D+HSD720D+HSD730D+HSD740D+HSD750D+HSD760D+HSD770D+HSD780D+HSG040D+HSG045D+HSG050D+HSG182D+HSG210D+HSG230D+HSG240D+HSG250D+HS

In [65]:
# OLS clustering at the County level
control_ols = smf.ols(control_formula , data=data).fit().get_robustcov_results(cov_type = "cluster", groups= data['CountyCode'])
control_ols_table = control_ols.summary2().tables[1]
control_ols_table

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\statsmodels\regression\linear_model.py:1817: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 196, but rank is 15
  warnings.warn('covariance of constraints does not have full '


,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,5.502326e-18,2.511469e-10,2.190879e-08,1.000000,-4.953289e-10,4.953290e-10
logfssl,1.931166e-01,5.295011e-02,3.647142e+00,0.000341,8.868477e-02,2.975483e-01
logrobr,1.876303e-01,5.418087e-02,3.463036e+00,0.000657,8.077114e-02,2.944895e-01
logburg,2.152371e-01,6.284583e-02,3.424843e+00,0.000751,9.128832e-02,3.391859e-01
burg_missing,1.499095e+00,4.455721e-01,3.364427e+00,0.000924,6.203073e-01,2.377882e+00
...,...,...,...,...,...,...
SPR130D,-1.614358e+00,8.617588e-01,-1.873329e+00,0.062527,-3.313976e+00,8.526130e-02
SPR230D,1.435790e+00,8.404060e-01,1.708448e+00,0.089153,-2.217152e-01,3.093296e+00
SPR330D,-6.187365e-02,4.048204e-01,-1.528422e-01,0.878682,-8.602878e-01,7.365405e-01
SPR440D,-4.500305e-01,2.718018e-01,-1.655731e+00,0.099393,-9.860963e-01,8.603530e-02


In [66]:
# Print results 
control_ols_table.loc[ 'logfssl' , : ]

Coef.       0.193117
Std.Err.    0.052950
t           3.647142
P>|t|       0.000341
[0.025      0.088685
0.975]      0.297548
Name: logfssl, dtype: float64

After controlling for a rich set of characteristics, the point estimate of gun ownership reduces to $0.19$.

# DML algorithm

Here we perform inference of the predictive coefficient $\beta$ in our partially linear statistical model, 

$$
Y = D\beta + g(Z) + \epsilon, \quad E (\epsilon | D, Z) = 0,
$$

using the **double machine learning** approach. 

For $\tilde Y = Y- E(Y|Z)$ and $\tilde D= D- E(D|Z)$, we can write
$$
\tilde Y = \alpha \tilde D + \epsilon, \quad E (\epsilon |\tilde D) =0.
$$

Using cross-fitting, we employ modern regression methods
to build estimators $\hat \ell(Z)$ and $\hat m(Z)$ of $\ell(Z):=E(Y|Z)$ and $m(Z):=E(D|Z)$ to obtain the estimates of the residualized quantities:

$$
\tilde Y_i = Y_i  - \hat \ell (Z_i),   \quad \tilde D_i = D_i - \hat m(Z_i), \quad \text{ for each } i = 1,\dots,n.
$$

Finally, using ordinary least squares of $\tilde Y_i$ on $\tilde D_i$, we obtain the 
estimate of $\beta$.

The following algorithm comsumes $Y, D, Z$, and a machine learning method for learning the residuals $\tilde Y$ and $\tilde D$, where the residuals are obtained by cross-validation (cross-fitting). Then, it prints the estimated coefficient $\beta$ and the corresponding standard error from the final OLS regression.

In [67]:
# Create main variables
Y = data['logghomr']
D = data['logfssl']
Z = data.drop(['logghomr', 'logfssl', 'CountyCode'], axis=1)
CLU = data['CountyCode']

In [68]:
# as matrix
y = Y.to_numpy().reshape( len(Y) , 1 )
d = D.to_numpy().reshape( len(Y) , 1 )
z = Z.to_numpy()
clu = CLU.to_numpy().reshape( len(Y) , 1 )

In [72]:
def DML2_for_PLM(z, d, y, dreg, yreg, nfold, clu):
    
    # Num ob observations
    nobs = z.shape[0]
    
    # Define folds indices 
    list_1 = [*range(0, nfold, 1)]*nobs
    sample = np.random.choice(nobs,nobs, replace=False).tolist()
    foldid = [list_1[index] for index in sample]

    # Create split function(similar to R)
    def split(x, f):
        count = max(f) + 1
        return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

    # Split observation indices into folds 
    list_2 = [*range(0, nobs, 1)]
    I = split(list_2, foldid)
    
    # loop to save results
    for b in range(0,len(I)):
    
        # Split data - index to keep are in mask as booleans
        include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
        mask = np.array([(i in include_idx) for i in range(len(z))])

        # Lasso regression, excluding folds selected 
        dfit = dreg(z[~mask,], d[~mask,])
        yfit = yreg(z[~mask,], y[~mask,])

        # predict estimates using the 
        dhat = dfit.predict( z[mask,] )
        yhat = yfit.predict( z[mask,] )

        # Create array to save errors 
        dtil = np.zeros( len(z) ).reshape( len(z) , 1 )
        ytil = np.zeros( len(z) ).reshape( len(z) , 1 )

        # save errors  
        dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
        ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
        print(b, " ")
    
    # Create dataframe 
    data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil,clu ), axis = 1), columns = ['ytil','dtil','CountyCode'])
   
    # OLS clustering at the County level
    model = "ytil ~ dtil"
    baseline_ols = smf.ols(model , data=data).fit().get_robustcov_results(cov_type = "cluster", groups= data_2['CountyCode'])
    coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
    se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']

    print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
    return coef_est, se, dtil, ytil
    

# Lasso 

In [92]:
def dreg(z,d):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, d)
    return result

def yreg(z,y):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, y)
    return result

DML2_lasso = DML2_for_PLM(z, d, y, dreg, yreg, 2, clu)

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11.513946376611688, tolerance: 0.0027673011711160734
  model = cd_fast.enet_coordinate_descent(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 129.34411382284642, tolerance: 0.0333156571571827
  model = cd_fast.enet_coordinate_descent(


0  


C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.724566223274573, tolerance: 0.0026042710571618087
  model = cd_fast.enet_coordinate_descent(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 139.67235980702853, tolerance: 0.03583729010907005
  model = cd_fast.enet_coordinate_descent(


1  
Coefficient is 0.20698020794932573, SE is equal to 0.07197871813280754


array([[ 0.09612708],
       [ 0.08080937],
       [ 0.05733992],
       ...,
       [ 0.17744358],
       [-0.09712691],
       [ 0.27406158]])

In [77]:
#DML with cross-validated Lasso:
# Reshaping Y variable
# Scalar distribution
scaler = StandardScaler()
scaler.fit( y )
std_y = scaler.transform( y )

scaler = StandardScaler()
scaler.fit( d )
std_d = scaler.transform( d )

In [78]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = LassoCV(cv = 10 , random_state = 0 , normalize = True ).fit( z, d )
    return result

def yreg(z,y):
    result = LassoCV(cv = 10 , random_state = 0 , normalize = True ).fit( z, y )
    return result

DML2_lasso_cv = DML2_for_PLM(z, std_d, std_y, dreg, yreg, 2, clu)

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.22541524177358951, tolerance: 0.16217056037393188
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.18034442593489075, tolerance: 0.16217056037393188
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\li

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 102.59735787235059, tolerance: 0.16217056037393188
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 119.1771696897722, tolerance: 0.16217056037393188
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 141.82922265870968, tolerance: 0.16217056037393188
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\env

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 39.44824087961524, tolerance: 0.15983510107304275
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 48.906712557169726, tolerance: 0.15983510107304275
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 70.75929614516986, tolerance: 0.15983510107304275
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 99.18473129658958, tolerance: 0.16012054879232734
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 105.46214383739095, tolerance: 0.16012054879232734
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 113.88368778211532, tolerance: 0.16012054879232734
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\env

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91.27370474410247, tolerance: 0.163946813617535
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 101.36796669736418, tolerance: 0.163946813617535
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 119.63335661471774, tolerance: 0.163946813617535
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tens

...

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 86.8928740924921, tolerance: 0.15881265460504748
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 81.66713679402983, tolerance: 0.15881265460504748
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 78.07652857353082, tolerance: 0.15881265460504748
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\t

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 47.165130901621296, tolerance: 0.20682684106769775
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 65.09735293083531, tolerance: 0.20682684106769775
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 82.96638384178709, tolerance: 0.20682684106769775
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.850319550507493, tolerance: 0.20249468860033915
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.800158069235977, tolerance: 0.20249468860033915
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11.144354263151627, tolerance: 0.20249468860033915
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4823395151142904, tolerance: 0.2046131659063678
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0707146261809157, tolerance: 0.2046131659063678
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.7778047555377725, tolerance: 0.2046131659063678
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\

KeyboardInterrupt: 

In [ ]:
# DML with cross-validated Lasso:
def dreg(z,d):
    result = ElasticNetCV( cv = 10 , normalize = True , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ).fit( z, d )
    return result

def yreg(z,y):
    result = ElasticNetCV( cv = 10 , normalize = True , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ).fit( z, y )
    return result

DML2_elnet = DML2_for_PLM(z, std_d, std_y, dreg, yreg, 2, clu)

In [ ]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = ElasticNetCV( cv = 10 , normalize = True , random_state = 0 , l1_ratio = 0.0001 ).fit( z, d )
    return result

def yreg(z,y):
    result = ElasticNetCV( cv = 10 , normalize = True , random_state = 0 , l1_ratio = 0.0001 ).fit( z, y )
    return result

DML2_ridge = DML2_for_PLM(z, std_d, std_y, dreg, yreg, 2, clu)

Here we also compute DML with OLS used as the ML method

In [ ]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = LinearRegression().fit( z, d )
    return result

def yreg(z,y):
    result = LinearRegression().fit( z, y )
    return result

DML2_ols = DML2_for_PLM(z, d, y, dreg, yreg, 2, clu)

Next, we also apply Random Forest for comparison purposes.

### Random Forest


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = RandomForestRegressor( random_state = 0 ).fit( z, d )
    return result

def yreg(z,y):
    result = RandomForestRegressor( random_state = 0 ).fit( z, y )
    return result

DML2_RF = DML2_for_PLM(z, d, y, dreg, yreg, 2, clu)   # set to 2 due to computation time

We conclude that the gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% relative
to a trend then the predicted gun homicide rate goes up by about 0.20% controlling for counties' characteristics.

Finally, let's see which method is actually better. We compute RMSE for predicting D and Y, and see which
of the methods works better.


In [ ]:
mods = ["DML2_ols", "DML2_lasso", "DML2_lasso_cv", "DML2_ridge", "DML2_elnet", "DML2_RF"]
mods_1 = pd.DataFrame(mods)
mods_1

In [ ]:
def mdl(model):
    RMSEY = np.sqrt(np.mean(model[3])**2) # I have some doubts about these equations...we have to recheck
    RMSED = np.sqrt(np.mean(model[2])**2) # I have some doubts about these equations...we have to recheck
    return RMSEY, RMSED

RES = mdl
    

In [ ]:
prRES_Y =  [RES(DML2_ols)[0],
RES(DML2_lasso)[0],
RES(DML2_lasso_cv)[0],
RES(DML2_ridge)[0],
RES(DML2_elnet)[0],
RES(DML2_RF)[0]]

In [ ]:
prRES_Y

In [ ]:
prRES_D = [RES(DML2_ols)[1],
RES(DML2_lasso)[1], 
RES(DML2_lasso_cv)[1], 
RES(DML2_ridge)[1],
RES(DML2_elnet)[1], 
RES(DML2_RF)[1]]

In [ ]:
prRES_D

It looks like the best method for predicting D is Lasso, and the best method for predicting Y is CV Ridge.

In [98]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = LassoCV(cv = 10 , random_state = 0 , normalize = True, alphas = [0]).fit( z, d )
    return result


def yreg(z,y):
    result = ElasticNetCV( cv = 10 , normalize = True , random_state = 0 , l1_ratio = 0.0001 ).fit( z, y )
    return result

DML2_best = DML2_for_PLM(z, std_d, std_y, dreg, yreg, 10, clu)

C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1390.1770294449234, tolerance: 0.32551778045292007
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent 

0  


C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1357.6389875061614, tolerance: 0.3150932306831086
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent w

1  


C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1434.9838644509791, tolerance: 0.33031034543425786
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent 

...

7  


C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1458.8959690210602, tolerance: 0.338568236733287
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent wi

8  


C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1455.4606914487074, tolerance: 0.3364015318748735
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent w

9  
Coefficient is 0.02731479764773729, SE is equal to 0.05641253624948492


In [ ]:
# def DML2_for_PLM(z=z, d=d, y=y, dreg=dreg, yreg=yreg, nfold = 10, clu=clu):
    
#     # Num ob observations
#     nobs = z.shape[0]
    
#     # Define folds indices 
#     list_1 = [*range(0, nfold, 1)]*nobs
#     sample = np.random.choice(nobs,nobs, replace=False).tolist()
#     foldid = [list_1[index] for index in sample]

#     # Create split function(similar to R)
#     def split(x, f):
#         count = max(f) + 1
#         return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

#     # Split observation indices into folds 
#     list_2 = [*range(0, nobs, 1)]
#     I = split(list_2, foldid)
    
#     # loop to save results
#     for b in range(0,len(I)):
    
#         # Split data - index to keep are in mask as booleans
#         include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
#         mask = numpy.array([(i in include_idx) for i in range(len(z))])

#         # Lasso regression, excluding folds selected 
#         dfit = dreg(z[~mask,], d[~mask,])
#         yfit = yreg(z[~mask,], y[~mask,])

#         # predict estimates using the 
#         dhat = dfit.predict( z[mask,] )
#         yhat = yfit.predict( z[mask,] )

#         # Create array to save errors 
#         dtil = np.zeros( len(z) ).reshape( len(z) , 1 )
#         ytil = np.zeros( len(z) ).reshape( len(z) , 1 )

#         # save errors  
#         dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
#         ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
#         print(b, " ")
    
#     # Create dataframe 
#     data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil,clu ), axis = 1), columns = ['ytil','dtil','CountyCode'])
   
#     # OLS clustering at the County level
#     model = "ytil ~ dtil"
#     baseline_ols = smf.ols(model , data=data).fit().get_robustcov_results(cov_type = "cluster", groups= data_2['CountyCode'])
#     coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
#     se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']

#     print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
#     return coef_est, se 
    

In [ ]:
# for b in range(0,len(I)):
    
#     # Split data - index to keep are in mask as booleans
#     include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
#     mask = numpy.array([(i in include_idx) for i in range(len(z))])
    
#     # Lasso regression, excluding folds selected 
#     dfit = dreg(z[~mask,], d[~mask,])
#     yfit = yreg(z[~mask,], y[~mask,])
    
#     # predict estimates using the 
#     dhat = dfit.predict( z[mask,] )
#     yhat = yfit.predict( z[mask,] )

#     # Create array to save errors 
#     dtil = np.zeros( len(z) ).reshape( len(z) , 1 )
#     ytil = np.zeros( len(z) ).reshape( len(z) , 1 )
    
#     # save errors  
#     dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
#     ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
#     print(b, " ")

# data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil,clu ), axis = 1), columns = ['ytil','dtil','CountyCode'])
# # OLS clustering at the County level
# model = "ytil ~ dtil"
# baseline_ols = smf.ols(model , data=data).fit().get_robustcov_results(cov_type = "cluster", groups= data_2['CountyCode'])
# coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
# se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']

# print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
# #return coef_est, se, dtil, ytil, rfit

In [ ]:
# nfold = 2
# nobs = 50

# # Define folds indices 
# list_1 = [*range(0, nfold, 1)]*nobs
# sample = np.random.choice(50,50, replace=False).tolist()
# foldid = [list_1[index] for index in sample]

# # Create split function(similar to R)
# def split(x, f):
#     count = max(f) + 1
#     return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

# # Split observation indices into folds 
# list_2 = [*range(0, nobs, 1)]
# I = split(list_2, foldid)